In [ ]:
import time
import numpy as np
import pandas as pd

import requests
from bs4 import BeautifulSoup

import time

import asyncio
import aiohttp

In [ ]:
page_min = 5918
page_max = 8875

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.0.0 YaBrowser/25.6.0.0 Safari/537.36'
}

In [ ]:
my_range = range(page_min, page_max+1)

final_dict = {
    'title': [],
    'out_year': [],
    'genres': [],
    'country': [],
    'duration': [],
    'description': [],
    'url_movie': [],
    'image_movie': [],
    'rating_votes': [],
    'film_rating_kp': [],
    'film_rating_imdb': []
}

In [ ]:
async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

async def main(urls):
    async with aiohttp.ClientSession() as session:
        tasks = [fetch(session, url) for url in urls]
        results = await asyncio.gather(*tasks)
    return results

urls = [f'https://lux.kinogo.biz/page/{i+1}/' for i in range(2859)]  # Список URL
asyncio.run(main(urls))

for page in my_range:
    start_time = time.time()
    url = f'https://lux.kinogo.biz/page/{page}/'
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')
    # 0. Найдём все блоки shortstory
    blocks = soup.find_all("div", class_="shortstory")

    # 1. Вытаскиваем все h2 из этих блоков, названия фильмов
    final_dict['title'].extend([block.find("h2").get_text(strip=True) if block.find("h2") else None for block in blocks])

    # 2. Вытаскиваем год выхода фильма
    final_dict['out_year'].extend([block.find("div", class_="shortstory__info-wrapper").find("a").get_text(strip=True) if block.find("div", class_="shortstory__info-wrapper").find("a") else None for block in blocks])

    # 3. Вытаскиваем URL фильма
    final_dict['url_movie'].extend([block.find("a").get("href") if block.find("a") else None for block in blocks])

    # 4. Вытаскиваем URL постера
    final_dict['image_movie'].extend(['https://lux.kinogo.biz' + block.find("div", class_="shortstory__poster").find("img").get('data-src') if block.find("div", class_="shortstory__poster").find("img") else None for block in blocks])
    
    # 5. Вытаскиваем жанры фильма
    for block in blocks:
        genre_title_tag = block.find('b', string='Жанр:')
        if genre_title_tag:
            genre_tags = genre_title_tag.find_next_siblings('a')
            movie_genres = [item.get_text() for item in genre_tags if item.get_text() not in ['Фильмы', 'Новинки']]
            final_dict['genres'].append(movie_genres)
        else:
            final_dict['genres'].append(None)
    
    # 6. Вытаскиваем страну производства фильма
    for block in blocks:
        country_title_tag = block.find('b', string='Страна:')
        if country_title_tag:
            country_tags = country_title_tag.find_next_siblings('a')
            movie_country = [item.get_text() for item in country_tags]
            final_dict['country'].append(movie_country)
        else:
            final_dict['country'].append(None)

    # 7. Вытаскиваем продолжительность фильма
    for block in blocks:
        duration_title_tag = block.find('b', string='Продолжительность:')
        if duration_title_tag:
            final_dict['duration'].append(duration_title_tag.next_sibling.strip())
        else:
            final_dict['duration'].append(None)
    
    # 8. Вытаскиваем описание фильма
    for block in blocks:
        description_title_tag = block.find("div", class_="excerpt")
        if description_title_tag:
            final_dict['description'].append(description_title_tag.get_text(strip=True))
        else:
            final_dict['description'].append(None)

    # 9. Вытаскиваем рейтинг фильма самого сайта 
    for block in blocks:
        rating_tag = block.find("div", class_="rating__votes")
        if rating_tag:
            final_dict['rating_votes'].append(rating_tag.get_text(strip=True))
        else:
            final_dict['rating_votes'].append(None)

    # 10. Вытаскиваем рейтинг фильма КИНО-ПОИСК
    for block in blocks:
        rating_kp_tag = block.find("span", class_="kp")
        if rating_kp_tag:
            final_dict['film_rating_kp'].append(rating_kp_tag.get_text(strip=True))
        else:
            final_dict['film_rating_kp'].append(None)

    # 11. Вытаскиваем рейтинг фильма IMDB
    for block in blocks:
        rating_imdb_tag = block.find("span", class_="imdb")
        if rating_imdb_tag:
            final_dict['film_rating_imdb'].append(rating_imdb_tag.get_text(strip=True))
        else:
            final_dict['film_rating_imdb'].append(None)

    end_time = time.time() - start_time
    print(f'Парсинг страницы {page} занял {end_time:.2f} сек')

    if page % 100 == 0:
        current_df = pd.DataFrame(final_dict)
        current_df.to_csv('movies_dataset.csv', index=False)
    if page == page_max:
        current_df = pd.DataFrame(final_dict)
        current_df.to_csv('movies_dataset.csv', index=False)

# time.sleep(np.random.rand()+1)

Парсинг страницы 5918 занял 0.58 сек
Парсинг страницы 5919 занял 0.65 сек
Парсинг страницы 5920 занял 0.62 сек
Парсинг страницы 5921 занял 0.54 сек
Парсинг страницы 5922 занял 0.62 сек
Парсинг страницы 5923 занял 0.58 сек
Парсинг страницы 5924 занял 0.59 сек
Парсинг страницы 5925 занял 0.84 сек
Парсинг страницы 5926 занял 0.57 сек
Парсинг страницы 5927 занял 0.60 сек
Парсинг страницы 5928 занял 0.58 сек
Парсинг страницы 5929 занял 0.59 сек
Парсинг страницы 5930 занял 0.56 сек
Парсинг страницы 5931 занял 0.60 сек
Парсинг страницы 5932 занял 0.61 сек
Парсинг страницы 5933 занял 0.50 сек
Парсинг страницы 5934 занял 0.57 сек
Парсинг страницы 5935 занял 0.53 сек
Парсинг страницы 5936 занял 0.56 сек
Парсинг страницы 5937 занял 0.58 сек
Парсинг страницы 5938 занял 0.60 сек
Парсинг страницы 5939 занял 0.56 сек
Парсинг страницы 5940 занял 0.73 сек
Парсинг страницы 5941 занял 0.95 сек
Парсинг страницы 5942 занял 0.95 сек
Парсинг страницы 5943 занял 0.91 сек
Парсинг страницы 5944 занял 0.70 сек
П